# Hartree Fock
## Molekül-Orbitale
### MO-LCAO Beschreibung von $H_2$

Hinweis: Dieses Notebook orientiert sich an S. 55 ff. im Buch Modern Quantum Chemistry: Introduction to Advanced Electronic Structure Theory von Szabo und Ostlund

Das 1s-Orbital eines Wasserstoff-Atoms ist gegeben durch

$$\Phi({\bf r}) = (\zeta^3/\pi)^{1/2} \exp(-\zeta(|{\bf r} - {\bf R}|)$$

mit $\zeta = 1.0$

Dieses Orbital wird auch *Slater Orbital* genannt



Alternativ kann das 1s-Orbital durch eine Gauß-Funktion dargestellt werden.
Das hat den Vorteil, dass Integrale leichter berechnet werden können.

Ein Gauß-Orbital hat die Form

$$\Phi({\bf r}) = (2\alpha/\pi)^{3/4} \exp(-\alpha|{\bf r} - {\bf R}|^2)$$

Mit $\alpha = 1$


In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

### Aufgabe 1

Schreiben Sie eine Funktion slater(r, R), die zwei Vektoren r und R als Eingabe hat und einen Wert für Phi zurückgibt.


### Aufgabe 2

Implementieren Sie analog eine Funktion, die ein Gauß-Orbital berechnet

Wenn Sie beide Funktionen in der unteren Zelle implementiert haben, schreiben Sie test() am Ende, um sich beide Funktionen plotten zu lassen

In [ ]:
def slater(r, R):
    # Hier bitte Code einfügen
    # Hinweis: den Betrag von r - R berechnen Sie mithilfe von np.linalg.norm(r - R, axis=-1)
    return (1 / np.pi)**0.5 * np.exp(-np.linalg.norm(r - R, axis=-1))


def gaussian(r, R):
    # Hier bitte Code einfügen
    return (2 / np.pi)**0.75 * np.exp(-np.linalg.norm(r - R, axis=-1)**2)


def test():
    # Erzeugt 50 Werte für r von -2 bis 2
    r = np.linspace(-2, 2, 50)[:, np.newaxis]
    # Der Ursprung wird auf 0 gesetzt
    R = 0

    slat = slater(r, R)
    gauss = gaussian(r, R)

    plt.plot(r, slat, label="Slater Orbital")
    plt.plot(r, gauss, label="Gaussian Orbital")
    plt.legend()
    plt.show()


test()

Sie sollten nun die zwei Orbitale sehen.


### Aufgabe 3 - Normierung

Vergewissern Sie sich nun, dass beide Orbitale normiert sind, indem Sie die quadrierten Wellenfunktionen über drei Dimensionen integrieren:
$$ \int\limits_V \Phi^*({\bf r})\Phi({\bf r}) dV \approx \sum\limits_i \Phi^*({\bf r})\Phi({\bf r}) \Delta V$$
Gehen Sie dazu vor wie folgt:
  * Erzeugen Sie mithilfe von np.linspace drei Vektoren x_range, y_range und z_range mit Werten zwischen -5 und 5
  * Definieren Sie eine Variable *summe*. 
  Iterieren Sie mithilfe dreier for-Schleifen über alle Punkte in x-, y- und z-Richtung und addieren Sie für jede x-y-z-Kombination das Quadrat der Wellenfunktion an dem jeweiligen Punkt multipliziert mit dem Volumenelement zu *summe*

In [ ]:
x_range = y_range = z_range = np.linspace(-5, 5)

summe_gauss, summe_slater = 0, 0
R = 0
dV = (x_range[1] - x_range[0])**3

for x in x_range:
    for y in y_range:
        for z in z_range:
            r = np.array([x, y, z])
            gauss_val = gaussian(r, R)
            slater_val = slater(r, R)
            summe_gauss += gauss_val**2 * dV
            summe_slater += slater_val**2 * dV
print(summe_gauss, summe_slater)

### Aufgabe 4 - Orthogonalität

Überprüfen Sie nun die Orthogonalität der Gaußschen Wellenfunktion.
Im Falle von $H_2$ und dem Modell der Minimal Orbital - Linear Combination of Atomic Orbitals (MO-LCAO), existieren zwei Gaußsche Funktionen $\Phi_1$ und $\Phi_2$, deren Zentren sich bei $R_1$ respektive $R_2$ befinden.

Berechnen Sie das Überlappintegral $$ S_{12} = \int\limits_V \Phi_1^*({\bf r}, {\bf R_1}) \Phi_2({\bf r}, {\bf R_2}) dV $$

* für $R_1$ = $R_2$ = [0, 0, 0]
* für $R_1$ = [0, 0, 0], $R_2$ = [1, 0, 0]
* Optional: Plotten Sie den Verlauf von $S_{12}$ für verschiedene Werte von $R_2$


Konstruieren Sie nun die folgenden Funktionen:

$$\Psi_1 = \frac{1}{\sqrt{2(1 + S_{12})}}(\Phi_1 + \Phi_2)$$

$$\Psi_2 = \frac{1}{\sqrt{2(1 - S_{12})}}(\Phi_1 - \Phi_2)$$

* Zeigen Sie, dass $\Psi_1$ und $\Psi_2$ orthonormal zueinander sind.
* Plotten Sie die beiden Funktionen entlang der x-Achse

In [ ]:
def overlap(f1, f2):
    x_range = y_range = z_range = np.linspace(-5, 5)
    summe = 0
    dV = (x_range[1] - x_range[0])**3
    for x in x_range:
        for y in y_range:
            for z in z_range:
                r = np.array([x, y, z])
                val1, val2 = f1(r), f2(r)
                summe += val1 * val2 * dV
    return summe


def phi_at_zero(r):
    return gaussian(r, 0)


def phi_at_one(r):
    return gaussian(r, [1, 0, 0])

S_12 = overlap(phi_at_zero, phi_at_one)

def psi_1(r):
    return 1 / (np.sqrt(2 * (1 + S_12))) * (phi_at_zero(r) + phi_at_one(r))


def psi_2(r):
    return 1 / (np.sqrt(2 * (1 - S_12))) * (phi_at_zero(r) - phi_at_one(r))


#print(overlap(phi_at_zero, phi_at_zero))
print(overlap(psi_1, psi_2))